In [ ]:
#use niwrap3912 conda environment
import os
import yaspy
import scipy.io
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''
#just get the labeling data from the MATLAB file (scholtens2018neuroimage_economo.mat, which is provided by the paper)
#Data from this paper: "An MRI Von Economo – Koskinas atlas" (Scholtens et al., 2018)
#They provide a matlab file with the "regionDescriptions"... so since parcellation is much harder from their volumentric data and freesurefer file
#use the easier way to do..
#But before using this data, make sure to convert the regionDescriptions (which is in stirng array in the struct) to a "char" array 
#because the scipy.io cannot load the string array correctly as it is not supported in the scipy.io.loadmat function
#then you can load the field as it is.
#load matlab file scholtens2018neuroimage_economo.mat. 

economo_mat = scipy.io.loadmat('scholtens2018neuroimage_economo_tochar.mat')
region_descriptions = [str(desc) for desc in economo_mat['ECONOMO_43']['regionDescriptions'][0][0]]
#remove the empty characters in each desciption (meaning remove empty spaces in each description
region_descriptions = [desc.strip() for desc in region_descriptions]

#calculate cell size
#Whereas cell size was calculated according to [H mean x W mean ]; 
#with H mean (Height) = [H (min-max) /2] and W mean (Width) = [W (min-max) /2] per individual cortical layer and then averaged across all layers.
#H is layer1total_thickness_dome
#W is layer1total_thickness_wall )

#save the cell size data to a numpy array
cell_size_data = []
for layer in ['I','II','III','IV','V','VI']:
    layer_name = f"layer{layer}"
    cell_size = economo_mat['ECONOMO_43'][f'{layer_name}total_cell_content_cellsize'][0][0]
   # print(cell_size)    
    cell_size_data.append(cell_size)
#get cell_content_cellsize from each layer (I..V) e.g., layerItotal_cell_content_cellsize from the matlab economo_mat
#then average across all layers
average_cell_size = np.nanmean(cell_size_data,axis=1)
print(average_cell_size)
'''


In [ ]:
#once the region descriptions are loaded, get the von ek atlas (already mapped in the 32k fsLR space)
#from Foit et al,. 2022

#so this cell size file "vonEK_cell_size_conte69_32k_lh.func.gii" has 43 regions (left and right hemisphere)
#the von ek atlas has 44 regions per hemispheres (including ctx-lh-uknown or ctx-rh-unknown)
myatlas_base_path = "/Users/dennis.jungchildmind.org/Downloads/MYATLAS_package_new/"
von_ek_subpath = "maps/Surface/HCP_conte69/conte69_32k/gii/cytoarchitecture/"
von_ek_lh = "vonEK_cell_size_conte69_32k_lh.func.gii"
von_ek_rh = "vonEK_cell_size_conte69_32k_rh.func.gii"
von_ek_cell_size_lh = nib.load(os.path.join(myatlas_base_path, von_ek_subpath,von_ek_lh)).darrays[0].data
von_ek_cell_size_rh = nib.load(os.path.join(myatlas_base_path, von_ek_subpath,von_ek_rh)).darrays[0].data
#this cell size data should be the same as the oens from the matlab file (from the scholtens2018neuroimage_economo.mat)
#though it is not the correct way because it does not have the "label" information in the myatlas file,
#we can still estimate the parcellation based on the "unique" values associated at each vertex 090225 -DJ
parcel_index = np.unique(von_ek_cell_size_lh)
#set each vertex with the unique parcel index
von_ek_atlas = np.zeros(von_ek_cell_size_lh.shape)
for i, parcel in enumerate(parcel_index):
    von_ek_atlas[von_ek_cell_size_lh == parcel] = i


In [ ]:
#use the atlas from the enigmatoolbox 
#needed to download from https://github.com/MICA-MNI/ENIGMA/blob/master/enigmatoolbox/datasets/parcellations/economo_koskinas_conte69.csv
#load the economo_koskinas_conte69.csv file
# this is in 32k conte (fs_LR) space
import pandas as pd
file_path = "/Users/dennis.jungchildmind.org/OneDrive - Child Mind Institute/layer_project/cortical_layer/external/economo_enigamtoolbox"
file_name = "economo_koskinas_conte69.csv"

#load the file
economo_koskinas_atlas = pd.read_csv(os.path.join(file_path, file_name), header=None)
#first half is the left hemisphere and the second half is the right hemisphere
economo_koskinas_atlas_lh = economo_koskinas_atlas.iloc[:32492,:].to_numpy().flatten()
economo_koskinas_atlas_rh = economo_koskinas_atlas.iloc[32492:,:].to_numpy().flatten()


In [ ]:
print(economo_koskinas_atlas)

In [ ]:
def plot_brain(data2plot, surface_file, hemisphere='lh', title=None,
               cmap='jet', figsize=(6,4), output_dir=None, savefilename_prefix=None, do_percentile=False, save_files = True):
    """
    Plot brain atlas data on cortical surface
    
    Parameters:
    -----------
    data2plot : array
        Data to plot on the surface
    surface_file : str
        Path to surface file (.surf.gii)
    hemisphere : str, default 'lh'
        Hemisphere ('lh' or 'rh')
    title : str
        Title for the plot
    cmap : str, default 'jet'
        Colormap to use
    figsize : tuple, default (6,4)
        Figure size
    output_dir : str, optional
        Directory to save outputs. If None, no files are saved
    filename_prefix : str, default 'von_ek_atlas'
        Prefix for saved files
    
    Returns:
    --------
    fig : matplotlib figure
        The created figure
    """
    plotter = yaspy.Plotter(surface_file, hemi=hemisphere)
    overlay = plotter.overlay(data2plot, cmap=cmap)

    # Generate multi-view montage
    views = ['lateral','medial']
    screenshots = [plotter.screenshot(view) for view in views]
    montage = yaspy.montage([[screenshots[0]],[screenshots[1]]], pad=8)
    
    # Create and style figure
    fig, ax = plt.subplots(figsize=figsize)
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.imshow(montage)
    plt.title(title)
    plt.axis('off')
    if do_percentile:
        vmin = np.percentile(data2plot, 5)
        vmax = np.percentile(data2plot, 95)
    else:
        vmin = np.nanmin(data2plot)
        vmax = np.nanmax(data2plot)
    # Add colorbar
    cax = ax.inset_axes([0.8, 0, 0.50, 0.04])
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm.set_array([])
    cbar = plt.colorbar(sm, cax=cax, orientation="horizontal", extend='both')
    cbar.set_label('Parcel Index', rotation=0, labelpad=10)
    cbar.set_ticks([np.nanmin(data2plot), np.nanmax(data2plot)])
    cbar.ax.tick_params(labelsize="x-small")
    

    if save_files:
        # Save outputs if directory specified
        if output_dir is not None:
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            plt.savefig(f'{output_dir}/{savefilename_prefix}.png', dpi=300)
            np.save(f'{output_dir}/{savefilename_prefix}_32k_fs_LR.npy', data2plot)
    
    return fig

hemi = 'rh'
# Use the function
if hemi == 'lh':
    surface_file = '/Users/dennis.jungchildmind.org/Downloads/MYATLAS_package_new/cortical_surface/conte69_32k/S1200.L.pial_MSMAll.32k_fs_LR.surf.gii'
elif hemi == 'rh':
    surface_file = '/Users/dennis.jungchildmind.org/Downloads/MYATLAS_package_new/cortical_surface/conte69_32k/S1200.R.pial_MSMAll.32k_fs_LR.surf.gii'
data2plot = von_ek_atlas.copy()
datamask = von_ek_atlas > 0
data2plot[~datamask] = np.nan
fig = plot_brain(data2plot, surface_file, hemisphere=hemi,
                 output_dir='von_ek_atlas', savefilename_prefix='von_ek_atlas',save_files=False)

data2plot = economo_koskinas_atlas_rh.copy().astype(float)
datamask = economo_koskinas_atlas_rh > 0
data2plot[~datamask] = np.nan

# Create discrete colormap with 5 distinct colors
from matplotlib.colors import ListedColormap
discrete_colors = {
    'Agranular': '#3478F6',   # Blue
    'Frontal': '#FFB6FF',     # Pink
    'Parietal': '#66FF66',    # Green  
    'Polar': '#FFFF66',       # Yellow
    'Granular': '#9966FF'     # Purple
}
discrete_cmap = ListedColormap(discrete_colors.values())

fig2 = plot_brain(data2plot, surface_file, hemisphere=hemi,
                 output_dir='von_ek_atlas', savefilename_prefix='von_ek_atlas',save_files=False, do_percentile=False, cmap=discrete_cmap)


In [ ]:
#do analysis separate for lh and rh
lh_intensity_data_uncut = [];
rh_intensity_data_uncut = [];
lh_intensity_data_at_zero = [];#at midline
rh_intensity_data_at_zero = [];#at midline
lh_label_data = [];
rh_label_data = [];
lh_subject_names = [];
rh_subject_names = [];
layer = 'inf'
data_type = "diff"

base_path = "/Users/dennis.jungchildmind.org/Desktop/exvivo"

base_path_intensity = "/Users/dennis.jungchildmind.org/Desktop/exvivo_figures_norm_120um"
for subjects in os.listdir(base_path):
    #for now skip I54_confidence
    if subjects == 'I54_new_confidence':
        continue
    if subjects:
        for hemispheres in ['lh','rh']:
            intensity_file_path = os.path.join(base_path_intensity, subjects, f"{hemispheres}/intensity_norm_{data_type}_results.npz")

            if os.path.exists(intensity_file_path):
                # Load intensity data
                data = np.load(intensity_file_path, allow_pickle=True)
                params = data['params']
                dist_array = params.item()['dist_array']

                if data_type == 'diff':
                    #at 0.06 is where the zero should be* I checked with local file
                    at_zero = np.where(dist_array == 0.06)[0][0]#this is where the zero should be at for diff dtta
                else:
                    at_zero = np.where(dist_array == 0.06)[0][0]# 

                #
                ap_order = data['inf_ap_order']
                # Sort data from anterior-posterior order to the surface order
                surface_order = np.argsort(ap_order)

                tmp_dat = data[f'{layer}_{data_type}_intensity'];
                tmp_dat = tmp_dat[:,surface_order]
                if hemispheres == 'lh':
                    lh_intensity_data_uncut.append(tmp_dat)
                    lh_intensity_data_at_zero.append(tmp_dat[at_zero,:])
                    lh_subject_names.append(subjects.replace('_new_confidence',''))
                else:
                    rh_intensity_data_uncut.append(tmp_dat)
                    rh_intensity_data_at_zero.append(tmp_dat[at_zero,:])
                    rh_subject_names.append(subjects.replace('_new_confidence',''))



In [ ]:
#parcellate intensity data based on the von ek atlas
#first get the unique values in the von ek atlas
atlas = economo_koskinas_atlas_lh #von_ek_atlas or economo_koskinas_atlas_lh or economo_koskinas_atlas_rh (should be the same for the latter two)
unique_values = np.unique(atlas)
print(unique_values)


rh_intensity_data_at_zero_parcelled = np.zeros((rh_intensity_data_at_zero[0].shape[0], len(rh_intensity_data_at_zero)))
rh_intensity_var_at_zero_parcelled = np.zeros((rh_intensity_data_at_zero[0].shape[0], len(rh_intensity_data_at_zero)))
print(rh_intensity_data_at_zero_parcelled.shape)
#iterate for each file
for j, tmp in enumerate(rh_intensity_data_at_zero):
    print(tmp.shape)
    #iterate for each parcel
    for i, parcel in enumerate(unique_values):
        indices = np.where(atlas == parcel)[0]
        #grab the data first
        tmp_data = tmp[indices]
        #find standard deviation of the data
        tmp_std = np.nanstd(tmp_data)
        #average the data but for the data that's within 2 std
        tmp_data_within_2std = tmp_data[tmp_data < tmp_std*2]
        tmp_mean = np.nanmean(tmp_data_within_2std)
        tmp_var = np.nanvar(tmp_data_within_2std)
        rh_intensity_data_at_zero_parcelled[indices,j] = tmp_mean
        rh_intensity_var_at_zero_parcelled[indices,j] = tmp_var



lh_intensity_data_at_zero_parcelled = np.zeros((lh_intensity_data_at_zero[0].shape[0], len(lh_intensity_data_at_zero)))
lh_intensity_var_at_zero_parcelled = np.zeros((lh_intensity_data_at_zero[0].shape[0], len(lh_intensity_data_at_zero)))
print(lh_intensity_data_at_zero_parcelled.shape)
#iterate for each file
for j, tmp in enumerate(lh_intensity_data_at_zero):
    print(tmp.shape)
    #iterate for each parcel
    for i, parcel in enumerate(unique_values):
        indices = np.where(atlas == parcel)[0]
        #grab the data first
        tmp_data = tmp[indices]
        #find standard deviation of the data
        tmp_std = np.nanstd(tmp_data)
        #average the data but for the data that's within 2 std
        tmp_data_within_2std = tmp_data[tmp_data < tmp_std*2]
        tmp_mean = np.nanmean(tmp_data_within_2std)
        tmp_var = np.nanvar(tmp_data_within_2std)
        lh_intensity_data_at_zero_parcelled[indices,j] = tmp_mean
        lh_intensity_var_at_zero_parcelled[indices,j] = tmp_var



In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

plot_individual_data = True
plot_hemi = 'lh'
data_type = 'mean' #'mean' or 'var'
cmap = 'viridis'
#atlas = von_ek_atlas 
# Surface files and settings
lh_surf = '/Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.white_MSMAll.32k_fs_LR.surf.gii'
rh_surf = '/Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.R.white_MSMAll.32k_fs_LR.surf.gii'


def mask_data(data, atlas_data):
    data_masked = data.copy()
    data_masked[atlas_data == 0] = np.nan
    return data_masked

# Select data based on hemisphere
if plot_hemi == 'lh':
    if data_type == 'mean':
        data_parc = lh_intensity_data_at_zero_parcelled
        clabel = 'Mean (intensity diff)'
    elif data_type == 'var':
        data_parc = lh_intensity_var_at_zero_parcelled
        clabel = 'Var (intensity diff)'
    surf2plot = lh_surf
    subject_names = lh_subject_names
elif plot_hemi == 'rh':
    if data_type == 'mean':
        data_parc = rh_intensity_data_at_zero_parcelled
        clabel = 'Mean (intensity diff)'
    elif data_type == 'var':
        data_parc = rh_intensity_var_at_zero_parcelled
        clabel = 'Var (intensity diff)'
    surf2plot = rh_surf
    subject_names = rh_subject_names

num_cols = lh_intensity_data_at_zero_parcelled.shape[1]#use the one with the higher number of sujbects (lh or rh)

#just create one plotter and reuse when overlay
plotter = yaspy.Plotter(surf2plot, hemi=plot_hemi)

if plot_individual_data:
    #perform standardization on the data_parc before averaging using zscore
    from scipy.stats import zscore
    data_parc_zscore = zscore(data_parc, axis=0, nan_policy='omit')
    data2plot_avg = np.nanmean(data_parc_zscore,axis=1)
    data2plot_sub = data_parc
    num_sub = data2plot_sub.shape[1]
    # Create figure with 2 rows (lateral/medial) and 1 + num_sub columns (avg + 11 raw)
    fig, axes = plt.subplots(2, 1+num_cols, figsize=(30, 4))
    
    
    # Calculate global min/max for this key across all data points
    all_data = np.concatenate([data2plot_avg[data2plot_avg != 0].flatten()])
    data_min = -1#np.nanpercentile(all_data[~np.isnan(all_data)], 5)
    data_max = 1#np.nanpercentile(all_data[~np.isnan(all_data)], 95)
# 
    # Plot average data (first column)
    data_masked = mask_data(data2plot_avg, atlas)
    
    for view_idx, view in enumerate(['lateral', 'medial']):
        #plotter = yaspy.Plotter(surf2plot, hemi=plot_hemi)
        #calculate data_min and datA_max of this data only
        plotter.overlay(data_masked, cmap=cmap, alpha=1,
                    vmin=data_min, vmax=data_max)
        plotter.border(data_masked, alpha=0)
        img = plotter.screenshot(view)
        
        axes[view_idx,0].imshow(img)
        axes[view_idx,0].axis('off')
        
        if view_idx == 0:
            axes[view_idx,0].set_title(f"\n(Average)", fontweight='bold',fontsize=16)
        else:
            divider = make_axes_locatable(axes[view_idx,0])
        #remove the colorbar for the averaged zscored data
        ''''
        else:
            divider = make_axes_locatable(axes[view_idx,0])
            cax = divider.append_axes("bottom", size="5%", pad=0.05)
            norm = plt.Normalize(vmin=data_min, vmax=data_max)
            cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), 
                            cax=cax, orientation='horizontal')
            cbar.set_label(clabel,fontsize=16)
        '''

    # Plot raw data (remaining columns)
    for col in range(num_sub):  # Plot all raw data columns
        data_masked = mask_data(data2plot_sub[:,col], atlas[0:32492])
        # Calculate min/max for this individual subject only
        subject_data = data_masked[data_masked != 0]
        data_min = np.nanpercentile(subject_data[~np.isnan(subject_data)], 5)
        data_max = np.nanpercentile(subject_data[~np.isnan(subject_data)], 95)
        
        for view_idx, view in enumerate(['lateral', 'medial']):
            #plotter = yaspy.Plotter(surf2plot, hemi=plot_hemi)
            plotter.overlay(data_masked, cmap=cmap, alpha=1,
                        vmin=data_min, vmax=data_max)
            plotter.border(data_masked, alpha=0)
            img = plotter.screenshot(view)
            
            axes[view_idx,col+1].imshow(img)
            axes[view_idx,col+1].axis('off')
            
            if view_idx == 0:
                axes[view_idx,col+1].set_title(f"{subject_names[col]}", fontweight='bold',fontsize=16)
            else:
                divider = make_axes_locatable(axes[view_idx,col+1])
                cax = divider.append_axes("bottom", size="5%", pad=0.05)
                norm = plt.Normalize(vmin=data_min, vmax=data_max)
                cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), 
                                cax=cax, orientation='horizontal')
                cbar.set_label(clabel,fontsize=16)

    # Handle empty columns if num_cols > num_sub
    if num_cols > num_sub:
        for col in range(num_sub, num_cols):
            axes[0,col+1].axis('off')
            axes[1,col+1].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
print(economo_koskinas_atlas_lh)